In [18]:
import os
from tkinter import *
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk

# Global variables for image and zoom level
img = None
zoom_level = 1.0

# Function to resize image to fit within 800x600
def resize_to_fit(img, max_width=800, max_height=600):
    width_ratio = max_width / img.width
    height_ratio = max_height / img.height
    resize_ratio = min(width_ratio, height_ratio)
    new_width = int(img.width * resize_ratio)
    new_height = int(img.height * resize_ratio)
    return img.resize((new_width, new_height), Image.Resampling.LANCZOS)

# Function to display image
def display_image():
    global img, img_label, zoom_level, canvas

    if img is None:
        return

    img_resized = resize_to_fit(img)
    img_resized = img_resized.resize((int(img_resized.width * zoom_level), int(img_resized.height * zoom_level)), Image.Resampling.LANCZOS)
    img_tk = ImageTk.PhotoImage(img_resized)

    img_label.config(image=img_tk)
    img_label.image = img_tk
    canvas.config(scrollregion=canvas.bbox(ALL))

# Function to open an image file
def open_image():
    global img, zoom_level, canvas
    filepath = filedialog.askopenfilename(
        filetypes=[("Image Files", "*.png;*.jpg;*.jpeg;*.bmp;*.gif")]
    )
    if not filepath:
        return

    img = Image.open(filepath)
    zoom_level = 1.0
    display_image()
    window.title(f"Image Viewer - {os.path.basename(filepath)}")

# Function to zoom in or out
def zoom(event):
    global zoom_level

    scale = 1.0
    if event.delta > 0:
        scale = 1.1
    elif event.delta < 0:
        scale = 0.9

    zoom_level *= scale
    display_image()
    canvas.config(scrollregion=canvas.bbox(ALL))

if __name__ == "__main__":
    window = Tk()
    window.title("Image Viewer")
    window.wm_iconbitmap("2.ico")
    window.geometry("800x600")

    canvas = Canvas(window, bg="white")
    img_label = Label(canvas)
    img_label.grid(row=0, column=0)

    hbar = Scrollbar(window, orient=HORIZONTAL, command=canvas.xview)
    hbar.pack(side=BOTTOM, fill=X)
    vbar = Scrollbar(window, orient=VERTICAL, command=canvas.yview)
    vbar.pack(side=RIGHT, fill=Y)

    canvas.config(xscrollcommand=hbar.set, yscrollcommand=vbar.set)
    canvas.pack(side=LEFT, expand=True, fill=BOTH)

    canvas.create_window((0, 0), window=img_label, anchor="nw")

    menubar = Menu(window)
    filemenu = Menu(menubar, tearoff=0)
    filemenu.add_command(label="Open", command=open_image)
    filemenu.add_separator()
    filemenu.add_command(label="Exit", command=window.quit)
    menubar.add_cascade(label="File", menu=filemenu)
    window.config(menu=menubar)

    window.bind("<MouseWheel>", zoom)

    window.mainloop()
